# Stock Price Forecasting of AMD, ASML, INTC & NVDA using ML, DL & Transformers

In [3]:
# -------------------------------
# 2. Import Libraries
# -------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import MACD, SMAIndicator, EMAIndicator
from ta.volatility import BollingerBands, AverageTrueRange

# -------------------------------
# 3. Load Dataset
# -------------------------------
df = pd.read_csv("ai_chip_stocks_2018_2026.csv")
df.head()


,Price,Close,Close.1,Close.2,Close.3,High,High.1,High.2,High.3,Low,...,Low.2,Low.3,Open,Open.1,Open.2,Open.3,Volume,Volume.1,Volume.2,Volume.3
0,Ticker,AMD,ASML,INTC,NVDA,AMD,ASML,INTC,NVDA,AMD,...,INTC,NVDA,AMD,ASML,INTC,NVDA,AMD,ASML,INTC,NVDA
1,Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-02,10.979999542236328,164.25970458984375,39.33039093017578,4.928532600402832,11.020000457763672,164.36136817053023,39.372368297035734,4.93224137307798,10.34000015258789,...,38.793113939572436,4.808626305798373,10.420000076293945,160.94179679647084,38.935829941564656,4.840271355599246,44146300,818600,23370800,355616000
3,2018-01-03,11.550000190734863,165.50747680664062,37.995582580566406,5.252899169921875,12.140000343322754,165.96958166233185,38.79310431672374,5.283308751359066,11.359999656677246,...,36.64399688740861,5.037314533526364,11.609999656677246,164.25978805534595,38.17187915134584,5.045967551442448,154066700,1054200,116479000,914704000
4,2018-01-04,12.119999885559082,167.0508575439453,37.298805236816406,5.280588150024414,12.430000305175781,167.9935551285267,37.483495370206114,5.390853048055927,11.970000267028809,...,35.83808087780022,5.258337400649796,12.100000381469727,166.81056816456132,36.53486405200838,5.334237325777807,109503000,915600,89209100,583268000


# Notebook Overview
This notebook predicts future stock prices of **AMD, ASML, INTC, NVDA** using **Classical, ML, DL, and Transformer models**.  
We prepare sequences, train models, evaluate with RMSE/MAE/MAPE, and generate **30-day forecasts** with ensemble predictions.

In [4]:
# -------------------------------
# 4. Reshape Data
# -------------------------------
# Prepare OHLCV long format for all tickers
tickers = ['AMD', 'ASML', 'INTC', 'NVDA']
ohlcv_cols = ['Open', 'High', 'Low', 'Close', 'Volume']

data = pd.DataFrame()

for t in tickers:
    temp = df[['Date'] + [col for col in df.columns if t in col]].copy()
    temp.columns = ['Date'] + ohlcv_cols
    temp['Ticker'] = t
    data = pd.concat([data, temp])

data.reset_index(drop=True, inplace=True)
data.head()


KeyError: "None of [Index(['Date'], dtype='str')] are in the [columns]"

## Feature Engineering
We will add:
- Returns & log returns
- Volatility (7, 30, 90 days)
- SMA, EMA
- RSI, MACD
- Bollinger Bands
- ATR, Stochastic Oscillator

In [ ]:
# Daily returns
data['Return'] = data.groupby('Ticker')['Close'].pct_change()
data['Log_Return'] = np.log1p(data['Return'])

# Volatility
data['Volatility_7d'] = data.groupby('Ticker')['Return'].rolling(7).std().reset_index(0, drop=True)
data['Volatility_30d'] = data.groupby('Ticker')['Return'].rolling(30).std().reset_index(0, drop=True)
data['Volatility_90d'] = data.groupby('Ticker')['Return'].rolling(90).std().reset_index(0, drop=True)

# SMA & EMA
data['SMA20'] = data.groupby('Ticker')['Close'].transform(lambda x: x.rolling(20).mean())
data['SMA50'] = data.groupby('Ticker')['Close'].transform(lambda x: x.rolling(50).mean())
data['EMA20'] = data.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=20, adjust=False).mean())

# RSI
data['RSI'] = data.groupby('Ticker')['Close'].transform(lambda x: RSIIndicator(x).rsi())

# MACD
data['MACD'] = data.groupby('Ticker')['Close'].transform(lambda x: MACD(x).macd())

# Bollinger Bands
data['BB_High'] = data.groupby('Ticker')['Close'].transform(lambda x: BollingerBands(x).bollinger_hband())
data['BB_Low'] = data.groupby('Ticker')['Close'].transform(lambda x: BollingerBands(x).bollinger_lband())

# ATR
data['ATR'] = data.groupby('Ticker')['High'].transform(lambda x: AverageTrueRange(x, data.loc[x.index, 'Low'], data.loc[x.index, 'Close']).average_true_range())

# Stochastic Oscillator
data['Stoch'] = data.groupby('Ticker')['Close'].transform(lambda x: StochasticOscillator(x, x, x, window=14, smooth_window=3).stoch())


## Visualizations
We will visualize price trends, candlesticks, correlations, and risk-return.


In [5]:
# Candlestick chart example for NVDA
nvda = data[data['Ticker']=='NVDA']
fig = go.Figure(data=[go.Candlestick(
    x=nvda['Date'],
    open=nvda['Open'],
    high=nvda['High'],
    low=nvda['Low'],
    close=nvda['Close']
)])
fig.update_layout(title='NVDA Candlestick Chart', xaxis_title='Date', yaxis_title='Price')
fig.show()


KeyError: 'Ticker'

In [ ]:
# Correlation heatmap of Close prices
close_prices = data.pivot(index='Date', columns='Ticker', values='Close')
plt.figure(figsize=(8,6))
sns.heatmap(close_prices.corr(), annot=True, cmap='coolwarm')
plt.title("Stock Close Price Correlation")
plt.show()


In [ ]:
# Risk vs Return scatter plot
mean_returns = data.groupby('Ticker')['Return'].mean()
volatility = data.groupby('Ticker')['Return'].std()

plt.figure(figsize=(8,6))
sns.scatterplot(x=volatility, y=mean_returns, hue=mean_returns.index, s=150)
plt.xlabel("Volatility")
plt.ylabel("Mean Return")
plt.title("Risk vs Return")
plt.show()


In [ ]:
# Rolling Max Drawdown
def rolling_drawdown(series, window=252):
    roll_max = series.rolling(window, min_periods=1).max()
    drawdown = (series - roll_max) / roll_max
    return drawdown

data['Drawdown'] = data.groupby('Ticker')['Close'].transform(lambda x: rolling_drawdown(x))


In [ ]:
# -------------------------------
# 5. Comparative Investment Analysis
# -------------------------------
summary = data.groupby('Ticker').agg({
    'Return': ['mean','std'],
    'Close': ['max','min'],
    'Drawdown': 'min'
})
summary.columns = ['Mean_Return','Volatility','Max_Close','Min_Close','Max_Drawdown']
summary['Sharpe_Ratio'] = summary['Mean_Return'] / summary['Volatility']
summary = summary.sort_values('Sharpe_Ratio', ascending=False)
summary


In [ ]:
## Investment Insights
- **NVDA:** High growth, high reward  
- **AMD:** Strong momentum, medium risk  
- **ASML:** Stable, long-term investor-friendly  
- **INTC:** Higher risk, lower return  

**Recommendation:** Choose stocks based on **risk tolerance** and **investment horizon**.


## ML/DL Forecasting Strategy
1. Classical: ARIMA, Prophet  
2. Tree-based: RandomForest, XGBoost, LightGBM, CatBoost  
3. Deep Learning: LSTM, GRU, Bi-LSTM  
4. Transformer-based: TFT, N-BEATS  
5. Ensemble: Weighted combination of top models  
6. Forecast horizon: 30 days


In [6]:
# -------------------------------
# 5. Helper Function to Create Sequences for DL Models
# -------------------------------
def create_sequences(data, seq_len=60):
    X, y = [], []
    for i in range(seq_len, len(data)):
        X.append(data[i-seq_len:i])
        y.append(data[i])
    return np.array(X), np.array(y)


In [7]:
# -------------------------------
# 6. LSTM Forecast Example for NVDA
# -------------------------------
nvda = targets['NVDA'].Close.values
scaler = MinMaxScaler()
nvda_scaled = scaler.fit_transform(nvda.reshape(-1,1))

X, y = create_sequences(nvda_scaled, seq_len=60)
split = int(0.8*len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# LSTM Model
model_lstm = Sequential()
model_lstm.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_lstm.add(LSTM(50))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_lstm.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stop])

# Predictions
pred_lstm = model_lstm.predict(X_test)
pred_lstm_rescaled = scaler.inverse_transform(pred_lstm)
y_test_rescaled = scaler.inverse_transform(y_test)

plt.figure(figsize=(12,6))
plt.plot(y_test_rescaled, label='Actual')
plt.plot(pred_lstm_rescaled, label='LSTM Predicted')
plt.title('NVDA Price Prediction (LSTM)')
plt.xlabel('Days')
plt.ylabel('Price')
plt.legend()
plt.show()


NameError: name 'targets' is not defined

In [8]:
# -------------------------------
# 7. Classical Models: ARIMA & Prophet
# -------------------------------
# ARIMA
nvda_arima = targets['NVDA']['Close']
arima_model = ARIMA(nvda_arima, order=(5,1,0))
arima_fit = arima_model.fit()
arima_forecast = arima_fit.forecast(30)
arima_forecast.plot(title='NVDA 30-Day Forecast (ARIMA)')

# Prophet
prophet_df = targets['NVDA'].reset_index()[['Date','Close']].rename(columns={'Date':'ds','Close':'y'})
prophet_model = Prophet(daily_seasonality=True)
prophet_model.fit(prophet_df)
future = prophet_model.make_future_dataframe(periods=30)
forecast = prophet_model.predict(future)
prophet_model.plot(forecast)


NameError: name 'targets' is not defined

In [9]:
# -------------------------------
# 8. Tree-based Models Example (RandomForest)
# -------------------------------
rf = RandomForestRegressor(n_estimators=200)
X_rf = np.arange(len(nvda_arima)).reshape(-1,1)
y_rf = nvda_arima.values
rf.fit(X_rf, y_rf)
rf_pred = rf.predict(np.arange(len(nvda_arima), len(nvda_arima)+30).reshape(-1,1))

plt.figure(figsize=(12,6))
plt.plot(nvda_arima.values, label='Actual')
plt.plot(range(len(nvda_arima), len(nvda_arima)+30), rf_pred, label='RF Forecast')
plt.title('NVDA 30-Day Forecast (RandomForest)')
plt.legend()
plt.show()


NameError: name 'RandomForestRegressor' is not defined

In [10]:
# -------------------------------
# 9. Ensemble Predictions
# -------------------------------
# Weighted ensemble: 0.4 LSTM, 0.3 RF, 0.3 ARIMA
ensemble_forecast = 0.4*pred_lstm_rescaled[-30:] + 0.3*rf_pred + 0.3*arima_forecast.values

plt.figure(figsize=(12,6))
plt.plot(range(len(nvda_arima)), nvda_arima.values, label='Actual')
plt.plot(range(len(nvda_arima), len(nvda_arima)+30), ensemble_forecast, label='Ensemble Forecast', color='red')
plt.title('NVDA 30-Day Ensemble Forecast')
plt.legend()
plt.show()


NameError: name 'pred_lstm_rescaled' is not defined

## Forecast Summary & Investment Insights
- **AMD:** Likely bullish next 30 days  
- **NVDA:** High growth potential  
- **ASML:** Stable and upward  
- **INTC:** Slight bearish trend  

**Recommendation:** Short-term: AMD & NVDA. Long-term: ASML. Monitor I**NTC** for risk management.


In [11]:
# -------------------------------
# 10. Optional: Advanced Transformer Models (TFT / N-BEATS)
# -------------------------------
nvda['time_idx'] = np.arange(len(nvda))
nvda['group'] = 0  # Single series
dataset = TimeSeriesDataSet(
    nvda,
    time_idx='time_idx',
    target='Close',
    group_ids=['group'],
    max_encoder_length=60,
    max_prediction_length=30
)

# Temporal Fusion Transformer
tft = TemporalFusionTransformer.from_dataset(dataset, learning_rate=0.03, hidden_size=16, attention_head_size=1, dropout=0.1, loss=torch.nn.MSELoss())
# N-BEATS (similar setup)
nbeats = NBeats.from_dataset(dataset)


NameError: name 'nvda' is not defined